In [1]:
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation, Dropout
from keras_tqdm import TQDMNotebookCallback
import keras.callbacks

from sqlalchemy import create_engine
import json
import pickle

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [151]:
version = 'v2'

# R  W  111
# R  P  11
# G  W  111
# G  P  11
# H  W  111
# H  P  11

# category = 'R'
# category = 'G'
category = 'H'

bet_type = 'W'
# bet_type = 'P'

In [152]:
engine = create_engine('sqlite:///../../data/race.db')
conn = engine.connect()
conn
dfoo = pd.read_sql_table('race', conn)
print('{} races loaded!'.format(len(dfoo)))

12711 races loaded!


In [153]:
dfo = dfoo.loc[dfoo['race_type'].isin([category])]
print('{} {} races!'.format(len(dfo), category))

3236 H races!


In [154]:
dfo['results'] = dfo['results_data'].map(json.loads)
dfo['runners'] = dfo['runners_data'].map(json.loads)
dfo.head(3)

/Users/jaco/.pyenv/versions/3.6.1/envs/tabby/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/jaco/.pyenv/versions/3.6.1/envs/tabby/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,id,meeting_name,location,venue_mnemonic,race_type,meeting_date,race_number,race_name,race_start_time,race_status,race_distance,results_data,num_runners,runners_data,quinella,exacta,trifecta,first_four,results,runners
36,37,CHARLTON,VIC,M,H,2017-10-01,1,DNR LOGISTICS 3YO PACE MS,2017-10-01 02:14:00,Paying,2100,"[[1], [5], [3], [4]]",6,"[{""runnerName"": ""TORQUE TOTHE STARZ"", ""runnerN...",3.3,15.6,36.1,57.3,"[[1], [5], [3], [4]]","[{'runnerName': 'TORQUE TOTHE STARZ', 'runnerN..."
37,38,CHARLTON,VIC,M,H,2017-10-01,2,AGISTMENT KILMORE PACE MS,2017-10-01 02:48:00,Paying,1609,"[[1], [2], [8], [7]]",8,"[{""runnerName"": ""SUBTLE EMBRACE"", ""runnerNumbe...",4.3,10.5,33.8,186.1,"[[1], [2], [8], [7]]","[{'runnerName': 'SUBTLE EMBRACE', 'runnerNumbe..."
38,39,CHARLTON,VIC,M,H,2017-10-01,3,VALE ROD GRANT TROT MS,2017-10-01 03:18:00,Paying,2100,"[[5], [1], [6], [2]]",10,"[{""runnerName"": ""SHESASUNDON"", ""runnerNumber"":...",6.5,14.0,91.7,249.9,"[[5], [1], [6], [2]]","[{'runnerName': 'SHESASUNDON', 'runnerNumber':..."


In [155]:
# extract runners from races
data_all = pd.DataFrame()
for ri, race in dfo.iterrows():
    data_all = data_all.append(race['runners'])    
data_all.tail(3)

,P_pred,P_prob,W_pred,W_prob,barrierNumber,claimAmount,finishingPosition,fixedOdds,has_odds,num_runners,...,place_rank,place_scaled,riderDriverName,runnerName,runnerNumber,trainerName,win_odds,win_perc,win_rank,win_scaled
7,NaN,NaN,NaN,NaN,10,0,0,"{'returnWin': 41, 'returnWinOpen': 26, 'return...",True,9,...,7.0,0.036070,GREG MILES,LIFES A JOURNEY,6,G J MILES,31.3,0.031949,7.0,0.026976
8,NaN,NaN,NaN,NaN,10,0,1,"{'returnWin': 11, 'returnWinOpen': 9, 'returnP...",True,9,...,4.0,0.127884,MORGAN WOODLEY,COEUR DE WHITBY,7,L T LEE,11.7,0.085470,5.0,0.072167
9,NaN,NaN,NaN,NaN,10,0,1,"{'returnWin': 1.75, 'returnWinOpen': 2.6, 'ret...",True,9,...,1.0,0.255769,DYLAN EGERTON-GREE,JAMES COBURN,1,S C REED,1.9,0.526316,1.0,0.444399


In [156]:
# drop scratched
data = data_all.dropna(subset=['win_odds', 'place_odds'])
data = data[(data.win_odds > 0)]
data = data[(data.place_odds > 0)]
data.describe()

,P_pred,P_prob,W_pred,W_prob,barrierNumber,claimAmount,finishingPosition,num_runners,place_odds,place_perc,place_rank,place_scaled,runnerNumber,win_odds,win_perc,win_rank,win_scaled
count,21766.000000,21766.000000,21766.000000,21766.000000,30724.000000,30724.0,30724.000000,30724.000000,30724.000000,30724.000000,30724.000000,30724.000000,30724.000000,30724.000000,30724.000000,30724.000000,30724.000000
mean,0.276181,0.105210,0.095747,0.105210,0.835829,0.0,1.003743,10.160656,5.256601,0.349894,5.495020,0.105325,5.789676,24.822665,0.124835,5.557479,0.105325
std,0.210665,0.083245,0.135580,0.141096,3.661062,0.0,1.397877,2.782150,5.597673,0.241514,3.310678,0.076389,3.448822,29.688370,0.143261,3.315783,0.120488
min,0.000002,0.000001,0.000000,0.000000,0.000000,0.0,0.000000,1.000000,1.000000,0.006667,1.000000,0.001886,1.000000,1.040000,0.001600,1.000000,0.001349
25%,0.098748,0.037897,0.014941,0.016905,0.000000,0.0,0.000000,8.000000,2.000000,0.153846,3.000000,0.045967,3.000000,6.200000,0.031447,3.000000,0.026514
50%,0.213575,0.081186,0.041458,0.048551,0.000000,0.0,0.000000,10.000000,3.400000,0.294118,5.000000,0.085502,5.000000,14.200000,0.070423,5.000000,0.059613
75%,0.431278,0.160180,0.116952,0.133578,0.000000,0.0,2.000000,12.000000,6.500000,0.500000,8.000000,0.146995,8.000000,31.800000,0.161290,8.000000,0.136330
max,1.000000,1.000000,0.999997,1.000000,80.000000,0.0,4.000000,20.000000,150.000000,1.000000,20.000000,1.000000,20.000000,625.000000,0.961538,20.000000,1.000000


In [157]:
# get label data
Y = data['finishingPosition']

if bet_type == 'W':
    Y = (Y == 1)
elif bet_type == 'P':
    Y = (Y == 1) | (Y == 2) | ((Y == 3) & (data['num_runners'] >= 8))

#print(data['finishingPosition'].head(10))
Y = Y.astype(int)
# Y.head(10)
Y.describe()

count    30724.000000
mean         0.105585
std          0.307311
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max          1.000000
Name: finishingPosition, dtype: float64

In [158]:
xn = data['num_runners']
xwp = data['win_perc']
xws = data['win_scaled']
xwr = data['win_rank']
xpp = data['place_perc']
xps = data['place_scaled']
xpr = data['place_rank']
X = pd.concat([xn, xwp, xws, xwr, xpp, xps, xpr], axis=1)
X.describe()

,num_runners,win_perc,win_scaled,win_rank,place_perc,place_scaled,place_rank
count,30724.000000,30724.000000,30724.000000,30724.000000,30724.000000,30724.000000,30724.000000
mean,10.160656,0.124835,0.105325,5.557479,0.349894,0.105325,5.495020
std,2.782150,0.143261,0.120488,3.315783,0.241514,0.076389,3.310678
min,1.000000,0.001600,0.001349,1.000000,0.006667,0.001886,1.000000
25%,8.000000,0.031447,0.026514,3.000000,0.153846,0.045967,3.000000
50%,10.000000,0.070423,0.059613,5.000000,0.294118,0.085502,5.000000
75%,12.000000,0.161290,0.136330,8.000000,0.500000,0.146995,8.000000
max,20.000000,0.961538,1.000000,20.000000,1.000000,1.000000,20.000000


In [159]:
# For a single-input model with 2 classes (binary classification):

n = len(X.columns)
print('input dimension = {}'.format(n))

epochs = 500
print('epochs = {}'.format(epochs))

layer_1 = 50
layer_2 = 50

tag = '{}{}x{}{}'.format(category, layer_1, layer_2, bet_type)
print('tag = {}'.format(tag))
file_name = '/Users/jaco/code/tabby/each_way/{}/models/{}.h5'.format(version, tag)

try:
    model = load_model(file_name)
    print('model loaded')
except OSError:
    model = Sequential()
    model.add(Dense(layer_1, activation='relu', input_dim=n))
    model.add(Dense(layer_2, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    print('model created')

input dimension = 7
epochs = 500
tag = H50x50W
model loaded


In [160]:
# Train the model, iterating on the data in batches of 32 samples
tqdm = TQDMNotebookCallback()

tbCallBack = keras.callbacks.TensorBoard(
    log_dir='/Users/jaco/code/tabby/each_way/{}/summary/{}'.format(version, tag), 
    histogram_freq=0,
    write_graph=True,
    write_images=True)

model.fit(
    X.as_matrix(), 
    Y.as_matrix(),
    validation_split=0.2,
    shuffle=True,
    epochs=epochs,
    batch_size=32,
    verbose=0,
    callbacks=[tqdm, tbCallBack])

# creates a HDF5 file
model.save(file_name)
print('model saved')


model saved
